In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
data = pd.read_csv('/content/drive/MyDrive/walmart_product_reviews_dataset.csv')
data.head()

,crawled_at,verified,reviewed_at,reviewed_by,helpful_count,not_helpful_count,content,raw_content,average_rating,review_heading,product_url,review_page_url,rating
0,"07/02/2021, 01:56:13",False,"November 3, 2019",Greenmachine,0,0,"Great buy, always go with white dark colors me...","<div class=""review-text""><p>Great buy, always ...",NaN,High quality good prices$Bachelor pad,https://www.walmart.com/ip/142064014,https://www.walmart.com/reviews/product/142064014,5.0
1,"07/02/2021, 01:56:17",False,"June 4, 2020",Donna,2,0,Awesome product...love it,"<div class=""review-text""><p>Awesome product......",3.6,NaN,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,5.0
2,"07/02/2021, 01:56:17",True,"May 15, 2021",sherita,0,0,"Love it, easy to put up as well!!!!","<div class=""review-text""><p>Love it, easy to p...",3.6,NaN,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,5.0
3,"07/02/2021, 01:56:17",True,"April 30, 2021",LaLaLady2,0,0,These go up very easily and I love the cordles...,"<div class=""review-text""><p>These go up very e...",3.6,Great blinds at a great price.,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,5.0
4,"07/02/2021, 01:56:17",True,"May 11, 2021",Hope,0,0,"When using the rod to open the blind, the rod ...","<div class=""review-text""><p>When using the rod...",3.6,My mini blind,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,4.0


In [26]:
def preprocess_text(text):
    if isinstance(text, str):
        words = nltk.word_tokenize(text)
        words = [word for word in words if word.lower() not in stopwords.words('english')]
        return ' '.join(words)
    else:
        return ''

In [27]:
data['cleaned_text'] = data['content'].apply(preprocess_text)

In [28]:
def get_sentiment_polarity(text):
    if isinstance(text, str):
        analysis = TextBlob(text)
        # Assign sentiment labels based on polarity
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity < 0:
            return 'negative'
        else:
            return 'neutral'
    else:
        return 'neutral'

In [29]:
data['sentiment'] = data['cleaned_text'].apply(get_sentiment_polarity)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['sentiment'], test_size=0.2, random_state=42)


In [31]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [32]:
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

In [33]:
y_pred = classifier.predict(X_test_tfidf)

In [34]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.6644864341085271
              precision    recall  f1-score   support

    negative       1.00      0.00      0.01       297
     neutral       0.80      0.00      0.01      1093
    positive       0.66      1.00      0.80      2738

    accuracy                           0.66      4128
   macro avg       0.82      0.34      0.27      4128
weighted avg       0.72      0.66      0.53      4128

